In [80]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce,api
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [81]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(6854, 7)

In [82]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6839, 5)

In [83]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6860, 9)

fin empresas y comienzo de rescate de pacientes


In [84]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1620187, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [85]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1177047, 4)

In [86]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1177047, 12)

In [88]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [89]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353


In [90]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [91]:
#pacientes_empresas22.shape

In [92]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [93]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [94]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [95]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Vida Security"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [96]:
Pacientes_Archivo.head(10)

,Cog Convenio,Nombre Convenio,Poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,Dv Asegurado,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email
0,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,9,5788383-9,TITULAR,RODOLFO,GUZMAN BASTIDAS,1952-12-17,M,2023-04-01,2024-03-31,RODOLFO.GUZMAN@ITAU.CL
1,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,6,7844961-6,CONYUGE/CONVIVIENTE,MONICA,ROJAS MOYA,1955-05-04,F,2023-04-01,2024-03-31,NaN
2,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387720,...,5,6065814-5,TITULAR,FERNANDO,LUCERO LOPEZ,1956-12-12,M,2023-04-01,2024-03-31,FERNANDOLUCEROL@HOTMAIL.COM
3,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,3,6240791-3,TITULAR,JUANITA,MARCONI CERDA,1954-07-20,F,2023-04-01,2024-03-31,JUANITA.MARCONI@ITAU.CL
4,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,4,6348612-4,TITULAR,CARLOS,NORAMBUENA DONOSO,1955-07-03,M,2023-04-01,2024-03-31,CARLOS.NORAMBUENA@ITAU.CL
5,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,6,8868904-6,CONYUGE/CONVIVIENTE,ROSA,BAEZA FIERRO,1958-12-13,F,2023-04-01,2024-03-31,NaN
6,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387720,...,9,6373921-9,TITULAR,VERONICA,HIRSCHBERG JACARD,1961-10-31,F,2023-04-01,2024-03-31,VERONICA.HIRSCHBERG@GMAIL.COM
7,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,9,6383042-9,TITULAR,ALEJANDRO,LOPEZ ORELLANA,1956-03-09,M,2023-04-01,2024-03-31,ALE.MAXLOPEZ@GMAIL.COM
8,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,5,6447017-5,TITULAR,FILOMENA MARIA,SANTOS SIERRA,1951-09-04,F,2023-04-01,2024-03-31,FILOMENA.SANTOS@ITAU.CL
9,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,1,6475287-1,TITULAR,JOSE,LAGOS MUÑOZ,1955-11-24,M,2023-04-01,2024-03-31,JOSE.LAGOS_MUNOZ@ITAU.CL


In [97]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [98]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,Cog Convenio,Nombre Convenio,Poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email,rut
0,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,5788383-9,TITULAR,RODOLFO,GUZMAN BASTIDAS,1952-12-17,M,2023-04-01,2024-03-31,RODOLFO.GUZMAN@ITAU.CL,57883839
1,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,7844961-6,CONYUGE/CONVIVIENTE,MONICA,ROJAS MOYA,1955-05-04,F,2023-04-01,2024-03-31,NaN,78449616
2,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387720,...,6065814-5,TITULAR,FERNANDO,LUCERO LOPEZ,1956-12-12,M,2023-04-01,2024-03-31,FERNANDOLUCEROL@HOTMAIL.COM,60658145
3,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,6240791-3,TITULAR,JUANITA,MARCONI CERDA,1954-07-20,F,2023-04-01,2024-03-31,JUANITA.MARCONI@ITAU.CL,62407913
4,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,6348612-4,TITULAR,CARLOS,NORAMBUENA DONOSO,1955-07-03,M,2023-04-01,2024-03-31,CARLOS.NORAMBUENA@ITAU.CL,63486124


In [99]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head() 

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10696\485331713.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Cog Convenio,Nombre Convenio,Poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email,rut
0,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,5788383-9,TITULAR,RODOLFO,GUZMAN BASTIDAS,1952-12-17,M,2023-04-01,2024-03-31,RODOLFO.GUZMAN@ITAU.CL,57883839
1,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,7844961-6,CONYUGE/CONVIVIENTE,MONICA,ROJAS MOYA,1955-05-04,F,2023-04-01,2024-03-31,NaN,78449616
2,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387720,...,6065814-5,TITULAR,FERNANDO,LUCERO LOPEZ,1956-12-12,M,2023-04-01,2024-03-31,FERNANDOLUCEROL@HOTMAIL.COM,60658145
3,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,6240791-3,TITULAR,JUANITA,MARCONI CERDA,1954-07-20,F,2023-04-01,2024-03-31,JUANITA.MARCONI@ITAU.CL,62407913
4,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,6348612-4,TITULAR,CARLOS,NORAMBUENA DONOSO,1955-07-03,M,2023-04-01,2024-03-31,CARLOS.NORAMBUENA@ITAU.CL,63486124


In [100]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10696\2118328728.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10696\2118328728.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [101]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(8839, 35)

In [102]:
Pacientes_con_id.head()

,Cog Convenio,Nombre Convenio,Poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,True,0018c00002kAakmAAC,57883839,Rodolfo Guzman Bastidas,RUT,None,None,None,Hombre,None
1,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,True,0018c00002kAaknAAC,78449616,Monica Rojas Moya,RUT,None,None,None,Mujer,None
2,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387720,...,True,0018c00002kAakoAAC,60658145,Fernando Lucero Lopez,RUT,None,None,None,Hombre,None
3,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,True,0018c00002kAakpAAC,62407913,Juanita Marconi Cerda,RUT,None,None,None,Mujer,None
4,113,CARE ASSISTANCE Plan GOLD,283144,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,81826800,9,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR DE...,387724,...,True,0018c00002kAakqAAC,63486124,Carlos Norambuena Donoso,RUT,None,None,None,Hombre,None


In [103]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"].astype(str).str.upper()
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        'Varón':'Hombre'      
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"].apply(str).str.strip().replace("+","")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"




if Pais == 'Chile':        
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:    
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtsEAE'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
221,False,Veronica Elvira,Uribarri Finger,86194732,"veronica.uribarri@iisachile.cl,",1968-03-27,Mujer,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE


In [104]:
    
clinica_virtual = pd.DataFrame()    
clinica_virtual['rut'] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual['firstName'] = Pacientes_con_id['nombres'].str.title().str.strip()
clinica_virtual['lastName'] = Pacientes_con_id['apellidos'].str.title().str.strip()
clinica_virtual['accountId'] = campaña



Codigos_Clinica_Virtual = {
'BCI':'1',
'Farmacias Ahumada':'2',
'MetLife':'3',
'Vida Security':'4',
'Willis':'5',
'Entel':'6',
'Testing':'7',
'3SP':'8',
'HomeCare':'9',
'Pedro de Valdivia':'10',
'Colaboradores':'11',
'MineraAbra':'12',
'CMPC':'13',
'Lipigas':'14',
'Asismed':'15',
'Siempre Perú':'16',
'Farmacias Cruz Verde':'17',
'WFS Food Services':'18',
'VidaCel':'19',
'Dermik':'20',
'EY':'21',
'IND':'22',
'Sindicato Walmart':'23',
'Instruvalve':'24',
'VidaCel Colaboradores':'25',
'Socofar':'26',
'Milan Fajanovic':'27',
'THBseguros':'28',
'Pesco':'29',
'Burgos - Consorcio':'30',
'COFAMA-SILENTIUM':'31',
'KAUFMANN':'32',
'QuePlan':'33',
'Zurich':'34',
'Colaboradores Siempre Chile':'35',
'Talo':'36',
'Transvip':'37',
'Vida Security Colaboradores':'38',
'FORUS':'39',
'SURA':'40',
'Southbrige':'41',
'Construmart':'42',
'BICE Vida':'43',
'Lockton':'44',
'Mercer':'45',
'Grupo PSG':'46'
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [105]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
390814,a1Y8c00000CldqXEAR,0018c00002azlRDAAY,a1W8c0000083rYiEAI,Vida Security,0018c00002axQfVAAU,S.Q.M. Industrial S.A.,Grupo S.Q.M.,Vida Security,S.Q.M. Industrial S.A.,283654,EMP6158,A-9176
391662,a1Y8c00000CleLsEAJ,0018c00002azmGuAAI,a1W8c0000083rYiEAI,Vida Security,0018c00002axQfVAAU,S.Q.M. Industrial S.A.,Grupo S.Q.M.,Vida Security,S.Q.M. Industrial S.A.,283654,EMP6158,A-9176
454532,a1Y8c00000Cm2JJEAZ,0018c00002b0RUiAAM,a1W8c0000083rZsEAI,Vida Security,0018c00002axQfOAAU,Minera Escondida Ltda.,Grupo BHP,Vida Security,Minera Escondida Ltda.,282349,EMP6151,A-9244
470506,a1Y8c00000CmDFMEA3,0018c00002bHw4eAAC,a1W8c0000083rZiEAI,Vida Security,0018c00002axQfKAAU,Empresa De Transporte Ferroviario S.A.,Ferronor,Vida Security,Empresa De Transporte Ferroviario S.A.,284254,EMP6147,A-9238
476378,a1Y8c00000CmGvvEAF,0018c00002bI3ErAAK,a1W8c0000083rYcEAI,Vida Security,0018c00002axQfEAAU,Emaresa Ingenieros Y Representaciones S.A.,Emaresa Ingenieros Y Representaciones S.A.,Vida Security,Emaresa Ingenieros Y Representaciones S.A.,283047,EMP6141,A-9170


In [106]:
campaña_seleccionada.shape

(90380, 12)

In [107]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [108]:
campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
campaña_en_pacientes = campaña_en_pacientes.rename(columns={'Programas_por_Empresas__c':'programas por empresa'})
campaña_en_pacientes['Paciente__c']=campaña_en_pacientes['Id Account Paciente']
campaña_en_pacientes['Programas_por_Empresas__c']=campaña_en_pacientes['programas por empresa']
campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [109]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
